# Demo 3 (LangChain, Agents)

We'll try to run a LangChain agent, able to interact with the world, perform a task
with a ["zero shot react"](https://python.langchain.com/docs/modules/agents/agent_types/react.html#using-zeroshotreactagent) approach.

We'll use the vanilla LangChain agent, but provide a set of custom tools to interact
with a custom world (a set of boxes containing items to move).

**Warning**: given the highly nondeterministic nature of this run, it might work or not,
it might arrive at a different correct end state than expected, it might raise errors
(and sometimes recover from it).

These are mostly limitations of the agent power rather than LangChain's. Here we just want
to illustrate a point about complex LLM usages and their ease with an LLM framework.

In [1]:
import os
from langchain.tools import BaseTool
from langchain.agents import AgentType, initialize_agent
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from pydantic import Field
from typing import Optional, Type

### Our custom "world"

In [2]:
class BoxStorage():
    def __init__(self, contents):
        """ contents is something like {box -> set-of-strings }"""
        self.boxes = {k.lower(): set(v) for k, v in contents.items()}
    
    def list_boxes(self): return list(self.boxes.keys())
    
    def list_box(self, box): return list(self.boxes[box])

    def move_objects(self, objs, bfrom, bto, log=False):
        assert bfrom in self.boxes
        assert bto in self.boxes
        if log:
            print(f'[move_objects] Trying to move {str(objs)} from {bfrom}[{str(list(self.boxes[bfrom]))}] to {bto}[{str(list(self.boxes[bto]))}]')
        for obj in objs:
            assert(obj in self.boxes[bfrom])
            self.boxes[bfrom] = self.boxes[bfrom] - {obj}
            self.boxes[bto] = self.boxes[bto] | {obj}

    def __repr__(self):
        return 'BoxStorage[%s]' % (', '.join(
            '%s=(%s)' % (k, ', '.join(sorted(v)))
            for k, v in sorted(self.boxes.items())
        ))

### Demonstration

In [3]:
test_storage = BoxStorage({"house": ["garbage", "kid", "couch"], "school": [], "sidewalk": ["car", "streetlight"]})

print(f"Start:\n    {test_storage}")

test_storage.move_objects(["kid"], "house", "school")

print(f"After bringing the kid to school:\n    {test_storage}")

test_storage.move_objects(["garbage"], "house", "sidewalk")

print(f"After taking the trash out:\n    {test_storage}")

Start:
    BoxStorage[house=(couch, garbage, kid), school=(), sidewalk=(car, streetlight)]
After bringing the kid to school:
    BoxStorage[house=(couch, garbage), school=(kid), sidewalk=(car, streetlight)]
After taking the trash out:
    BoxStorage[house=(couch), school=(kid), sidewalk=(car, garbage, streetlight)]


## Define the "tools"

These are classes wrapping functions with a special signature (and bound to an instance of the "storage").

We opt for simple string input and try to parse it in the tool code itself:

In [4]:
class CustomListBoxesTool(BaseTool):
    name = "list_boxes"
    description = "The tool to get a comma-separated list of available boxes. The input is ignored."
    storage: BoxStorage = Field(exclude=True)
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return ', '.join(self.storage.list_boxes())

def _clean(tkn):
    return tkn.replace(',', '').strip().lower()

class CustomMoveObjectsTool(BaseTool):
    name = "move_object"
    description = """This is the tool you use to physically move a space-separated list of objects from a box to another box.
    The input must have the form "MOVE object_name_1 object_name_2 ... FROM source_box TO destination_box".
    Each invocation can move a number of objects to exactly one box."""
    storage: BoxStorage = Field(exclude=True)
    
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        # an ugly syntax validation, lol
        qchunks = [_clean(c) for c in query.split(' ') if _clean(c)]
        fromIndex = [i for i, c in enumerate(qchunks) if c=='from'][0]
        assert(qchunks[0].lower() == 'move')
        assert(qchunks[fromIndex].lower() == 'from')
        assert(qchunks[2 + fromIndex].lower() == 'to')
        assert(len(qchunks) == 4 + fromIndex)
        obs_to_move = set(qchunks[1:fromIndex])
        bfrom = qchunks[1 + fromIndex]
        bto = qchunks[3 + fromIndex]
        self.storage.move_objects(obs_to_move, bfrom, bto)

class CustomListBoxContentTool(BaseTool):
    name = "box_contents"
    description = "The tool to get a comma-separated list of contents of a box. The input is the box name."
    storage: BoxStorage = Field(exclude=True)

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return ', '.join(self.storage.list_box(_clean(query)))


### The world starts like this:

In [5]:
storage = BoxStorage({'box1': {'apple','banana','pear'}, 'box2': {'car', 'bicycle', 'train'}})
print(f'At the beginning: {storage}')

At the beginning: BoxStorage[box1=(apple, banana, pear), box2=(bicycle, car, train)]


### A toolkit for the agent

In [6]:
tools = [
    CustomListBoxesTool(storage=storage),
    CustomListBoxContentTool(storage=storage),
    CustomMoveObjectsTool(storage=storage)
]

### Creating the agent

In [7]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI()

In [8]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

### Starting the agent with a task

In [9]:
agent.run("""
    You are given some named boxes, each containing some named items,
    and the tools to inspect and operate on them.
    Verify if there are pears or apples in box1,
    then move all of them to box2. You can move several objects at once,
    if that is permitted by the available tools.
    Stop when there are neither pears nor apples in box1.
""")



> Entering new AgentExecutor chain...
First, I need to check the contents of box1 to see if there are any pears or apples. If there are, I need to move them to box2. I will need to keep checking and moving until there are no more pears or apples in box1.

Action: list_boxes
Action Input: 

Observation: box1, box2
Thought:I have the list of boxes: box1 and box2. Now I need to check the contents of box1.

Action: box_contents
Action Input: box1

Observation: pear, banana, apple
Thought:I see that box1 contains pears and apples. I need to move them to box2. Let me do that.

Action: move_object
Action Input: MOVE pear apple FROM box1 TO box2

Observation: None
Thought:I have moved the pears and apples from box1 to box2. Now I need to check the contents of box1 again to see if there are any remaining pears or apples.

Action: box_contents
Action Input: box1

Observation: banana
Thought:I see that box1 now only contains a banana. There are no more pears or apples in box1, so I can stop now

'There are no more pears or apples in box1.'

In [10]:
print(f'At the end: {storage}')

At the end: BoxStorage[box1=(banana), box2=(apple, bicycle, car, pear, train)]
